In [1]:
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import sys
import time

from torch import nn

import asyncio ,nest_asyncio
from python_tools.discord_bot import async_discord_bot_notifier ,discord_bot_notifier
nest_asyncio.apply()


### control line

In [2]:
# csv_file = '../data/img_conv2.csv'
random_seed = 42
# dataset_path = "../data/HAM10000/images/"
# groundtruth_file = '../data/HAM10000/GroundTruth.csv'
# feature_vector_file_path = '../data/HAM10000/img_feature_no_masked.csv'

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

# model_select_signal = 'resnet18'
model_select_signal = 'vgg16'
# model_select_signal = 'densenet121'
# model_select_signal = 'feature vec'


data_select_signal = 'skin'
# data_select_signal = 'chest CT'
# data_select_signal = 'ocularDisease'

# select image type is RGB or not
isRGB = True
# isRGB = False

# enable img feature vector as mutimodal
# enable_muti_modal_signal = True
enable_muti_modal_signal = False

## dataset prepocessing

In [3]:
from helperFunction.CustomImageDataset import CustomImageDataset
import os
import random
import torchvision
import json

# 數據轉換
transform_std = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform224 = torchvision.transforms.Compose([
    transforms.Resize(size=(224,224)),
    torchvision.transforms.ToTensor()
    # 其他轉換
])

transform = transform_std


# all_files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]

# random.seed(42)

# num_total_samples = len(all_files)
# split_ratio = 0.8  # 80% 的数据用于训练，20% 用于测试

# # 随机打乱数据集
# random.shuffle(all_files)

# # 计算分割点
# split_idx = int(num_total_samples * split_ratio)

# # 分割数据集
# train_files = all_files[:split_idx]
# test_files = all_files[split_idx:]

# 載入train & test file list
# with open('test_files_list.json', 'r') as f:
#      test_files = json.load(f)
# with open('train_files_list.json', 'r') as f:
#      train_files = json.load(f)


# # 加載數據
# train_dataset = CustomImageDataset(img_dir=dataset_path,file_to_label_dict={file: filename_to_label_dict[file] for file in train_files}, transform=transform)
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# test_dataset = CustomImageDataset(img_dir=dataset_path,file_to_label_dict={file: filename_to_label_dict[file] for file in test_files}, transform=transform)
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
# @ transfrom test unit

# import unittest
# import numpy as np
# import torch
# from torchvision import transforms
# from PIL import Image

# class TestTransforms224Gray32bit(unittest.TestCase):
#     def setUp(self, img_path=None):
#         # 创建一个纯白色的测试图像，尺寸为 300x300
#         if img_path :
#             self.image = Image.open(img_path).convert('L')
#         else :
#             self.image = Image.new('L', (300, 300), color='white')

#     def test_transform_flow(self):
#         transform224_gray_32bit = transforms.Compose([
#             transforms.Lambda(lambda img: np.array(img).astype(np.float32) / 255.0),
#             transforms.Lambda(lambda x: torch.from_numpy(x)),
#             transforms.Resize(256),
#             transforms.CenterCrop(224),
#             transforms.Normalize(mean=[0.485], std=[0.229]),
#         ])
        
#         # 应用转换流程
#         transformed_img = transform224_gray_32bit(self.image)
        
#         # 检查转换后的类型和形状
#         self.assertTrue(isinstance(transformed_img, torch.Tensor), "Output should be a Torch Tensor")
#         self.assertEqual(transformed_img.dtype, torch.float32, "Output tensor should have dtype float32")
#         self.assertEqual(transformed_img.size(), (1, 224, 224), "Output tensor should have shape (1, 224, 224)")
        
#         # 检查值的范围是否合理（因为原图是纯白的，所以归一化后应该有一个固定的范围）
#         expected_value = (1 - 0.485) / 0.229
#         self.assertTrue(torch.allclose(transformed_img.mean(), torch.tensor(expected_value), atol=1e-5),
#                         "Normalized values do not match expected range")

# test = TestTransforms224Gray32bit()
# test.setUp('../data/chestCTData/images/train/adenocarcinoma/000000 (6).png')

# test.test_transform_flow()

In [5]:
sys.path.append(os.path.abspath('..'))


from data.HAM10000.ham10000Dataloader import HAM10000DataProcessor
from data.chestCTData.chestCTDataloader import ChestCTDataProcessor
from data.ocularDisease.ocularDataloader import OcularDiseaseDataProcessor

if data_select_signal == 'skin':
    dataContainer = HAM10000DataProcessor(transform=transform_std)
elif data_select_signal == 'chest CT':
    dataContainer = ChestCTDataProcessor(transform=transform_std)
elif data_select_signal == 'ocularDisease':
    dataContainer = OcularDiseaseDataProcessor(transform=transform_std)
else:
    raise ValueError('需要指定dataset類別')

train_dataloader , test_dataloader = dataContainer.getDataloaders()
train_files , test_files = dataContainer.getDatasetFilenames()
# feature_vector_file_path = dataContainer.getFeatureVectorFilename()

num_classes = dataContainer.getNumClasses()



In [6]:
print(type(transform224))

# print(train_files[0:5])

<class 'torchvision.transforms.transforms.Compose'>


### load color feature data base on dataloader filename idx

In [7]:
# from helperFunction.XgbHelperFunction import csvkeylistToData


# train_feature_vectors = csvkeylistToData(feature_vector_file_path, train_files)
# test_feature_vectors = csvkeylistToData(feature_vector_file_path, test_files)

### cal img feature vectors

In [8]:
from helperFunction.helperFunctions import dataloaderToFeatureData , calImgFeatureVector
train_img_feature_vector = []
test_img_feature_vector = []

def print_list_dimensions(lst):
    dimensions = []
    while isinstance(lst, list):
        dimensions.append(len(lst))
        lst = lst[0] if len(lst) > 0 else []
    print("Dimensions:", " x ".join(map(str, dimensions)))

    return


if enable_muti_modal_signal:

    for idx , (batch_data, label) in enumerate(train_dataloader):
        for img in batch_data:
            train_img_feature_vector.append(calImgFeatureVector(img, isRGB=isRGB))
    
    # print_list_dimensions(train_img_feature_vector)

    for idx , (batch_data, label) in enumerate(test_dataloader):
        for img in batch_data:
            test_img_feature_vector.append(calImgFeatureVector(img, isRGB=isRGB))
    
    # print_list_dimensions(test_feature_vector)
        
    # for idx , (data, label) in enumerate(test_dataloader):
    #     train_img_feature_vector.append( calImgFeatureVector(data, isRGB=isRGB) )

### define XGB eval recorder function

In [9]:
# define XGB training function
from helperFunction.helperFunctions import dataloaderToFeatureData , calImgFeatureVector
from helperFunction.XgbHelperFunction import  train_predict, calBestIterOfXGB, calFPGAFormatXGB
import xgboost as xgb


# 負責呼叫與紀錄結果
def recordXGBoutput(model:nn.Sequential, train_dataloader,
                    test_dataloader, enable_muti_module:bool=False,
                    isRGB:bool=False, model_output_path:str=None,
                    FPGA_format=False
                    ):
  print("cal CNN model output...")
  test_features , test_labels = dataloaderToFeatureData(model, test_dataloader,device)
  
  start_time = time.time()
  train_features , train_labels = dataloaderToFeatureData(model, train_dataloader, device)
  end_time = time.time()
  CNN_eval_time = (end_time - start_time)/len(train_features)

  # muti modal selection
  if enable_muti_module:

    train_muti_modal_vectors = []
    for idx , data in enumerate(train_features):
      train_muti_modal_vectors.append(np.concatenate((train_features[idx] , train_img_feature_vector[idx])))
    train_features = np.array(train_muti_modal_vectors)

    test_muti_modal_vectors = []
    for idx , data in enumerate(test_features):
      test_muti_modal_vectors.append(np.concatenate((test_features[idx] , test_img_feature_vector[idx])))
    test_features = np.array(test_muti_modal_vectors)



  print("feature size is : {}".format(test_features.shape))
  # print_list_dimensions(test_features)
  
  # print(feature for feature in test_features if len(feature) == 0)
  # nan_indices = np.where(np.isnan(test_features))
  # print(f'Indices of NaN values: {list(zip(nan_indices[0], nan_indices[1]))}')

  
  # xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=num_classes)

  # 呼叫自定義的訓練func
  if FPGA_format: # 呼叫符合FPGA中規定好的訓練
    XGB_result = calFPGAFormatXGB(train_features, train_labels, test_features, test_labels, device, enable_f1_metric=True, model_output_path=model_output_path)
  else : 
    XGB_result = calBestIterOfXGB(train_features, train_labels, test_features, test_labels, device, enable_f1_metric=True, model_output_path=model_output_path)

  # f1, acc = train_predict(xgb_model, train_features, train_labels,  test_features, test_labels)

  # 計算CNN model的大小
  total_size = 0
  for p in model.parameters():
    total_size += p.numel() * p.element_size()

  print("======eval finish!=========")
  XGB_result['CNN size'] = total_size
  XGB_result['CNN eval time'] = CNN_eval_time

  return XGB_result
  


### define ML eval recorder function

In [10]:
def recordMLoutput(ML_model, model:nn.Sequential, 
                   train_dataloader, test_dataloader,
                   enable_muti_module=False):
  """使用剪枝後的CNN model 的輸出值來訓練ML model"""

  print("cal CNN model output...")
  test_features , test_labels = dataloaderToFeatureData(model, test_dataloader,device)
  str_time = time.time()
  train_features , train_labels = dataloaderToFeatureData(model, train_dataloader, device)


  if enable_muti_module:
    train_muti_modal_vectors = []
    for idx , data in enumerate(train_features):
      train_muti_modal_vectors.append(np.concatenate((train_features[idx] , train_img_feature_vector[idx])))
    train_features = np.array(train_muti_modal_vectors)

    test_muti_modal_vectors = []
    for idx , data in enumerate(test_features):
      test_muti_modal_vectors.append(np.concatenate((test_features[idx] , test_img_feature_vector[idx])))
    test_features = np.array(test_muti_modal_vectors)


  nan_indices = np.where(np.isnan(test_features))

  
  # print(len(test_features[0]))
  f1, acc = train_predict(ML_model, train_features, train_labels,  test_features, test_labels)

  print("======eval finish!=========")

  return f1, acc 

## init nn module

### 初始化模型

In [11]:
from torchinfo import summary

if data_select_signal == 'skin':
    model_folder_path = "../model/HAM10000"
elif data_select_signal == 'chest CT':
    model_folder_path = "../model/CT chest"
elif data_select_signal == 'ocularDisease':
    model_folder_path = "../model/ocularDisease"



# resnet101 = models.resnet101(pretrained=True)
# #  ===================================
# # 加載預訓練的ResNet模型
# resnet18 = models.resnet18(pretrained=True)
# resnet18 = torch.nn.Sequential(*(list(resnet18.children())[:-1]))  # 移除最後的全連接層

#  ===================================
# 加載訓練好的ResNet模型
resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, num_classes)
resnet18.load_state_dict(torch.load(model_folder_path + "/best_model_pretrain_Resnet18.pth"))
# resnet18_7  = torch.nn.Sequential(*(list(resnet18_7.children())[:-1]))  # 移除最後的全連接層

# #  =========================
# resnet50 = models.resnet50(pretrained=True)
# num_ftrs = resnet50.fc.in_features
# resnet50.fc = nn.Linear(num_ftrs, num_classes)
# resnet50.load_state_dict(torch.load(model_folder_path + "/best_model_pretrain_Resnet50_7.pth"))

# #  ===================================
# # 加載預訓練的VGG模型
# vgg16 = models.vgg16(pretrained=True)
# vgg16.classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-1]) # 移除最後的全連接層

#  ===================================
# 載入訓練好的vgg
vgg16 = models.vgg16(pretrained=True)
classifier = list(vgg16.classifier.children())[:-1]

# 移除原始模型的最后一个全连接层
# 并添加一个新的全连接层，输出特征数为 輸出的種類數
classifier.append(torch.nn.Linear(4096, num_classes))

# 替换原始模型的分类器
vgg16.classifier = torch.nn.Sequential(*classifier)

vgg16.load_state_dict(torch.load(model_folder_path + "/best_model_pretrain_VGG16.pth"))

# 使用nn.Sequential的方式取代torch.flatten的功能
new_classfier = nn.Sequential(
    nn.Flatten(),
    vgg16.classifier,
)

vgg16.classifier = new_classfier

# vgg16.classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-1]) # 移除最後的全連接層

# ================================
# # 加載訓練好的densenet
# densenet121 = models.densenet121(pretrained=True)
# # Optimizer
# optimizer = torch.optim.SGD(densenet121.parameters(), lr = 0.001) # 選擇你想用的 optimizer
# # optimizer = torch.optim.Adam(model_densenet121.parameters(), lr =0.01)

# # Loss function
# loss_fn = nn.CrossEntropyLoss()                

# # 更換classifier的輸出
# densenet121.classifier = nn.Linear(densenet121.classifier.in_features, num_classes)


# densenet121.load_state_dict(torch.load(model_folder_path + "/best_model_pretrain_densenet121.pth"))
# ================================
# 設定空的model給feaure訓練用



class EmptyModel(nn.Module):
    def __init__(self):
        super(EmptyModel, self).__init__()

    def forward(self, x):
        # 返回一个空的张量
        return torch.empty((x.size(0), 0), dtype=torch.float32)

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`.

In [12]:
# 決定使用的模型
model_0 = None

if model_select_signal == 'resnet18':
    model_0 = resnet18
elif model_select_signal == 'vgg16':
    model_0 = vgg16
elif model_select_signal == 'feature vec':
    model_0 = nn.Sequential(EmptyModel())
elif model_select_signal == 'densenet121':
    model_0 = densenet121


model_0 = model_0.to(device)

summary(model_0, input_size=[1,3,224,224])

# get var name
model_0_name = [name for name, val in globals().items() if val == model_0][0]

In [13]:
# dataContainer.dataset_label_distribution()

In [14]:
# test_features , test_labels = dataloaderToFeatureData(model_0, test_dataloader,device)
# train_features , train_labels = dataloaderToFeatureData(model_0, train_dataloader, device)


# for idx , data in enumerate(train_features):
#     train_features[idx] = np.concatenate(train_features[idx] , train_img_feature_vector[idx])



In [15]:
# @ function test unit : dataloaderToFeatureData w/ img feature enhence
# train_features , train_labels = dataloaderToFeatureData(model_0, train_dataloader, device)

# print(len(train_features))
# print(type(train_features))
# print(train_features[0])

In [16]:
# @ unit test : find specific layers

# import torchvision.models as models

# model_0 = models.resnet18(pretrained=True)

# total_layers = 0

# for name, layer in model_0.named_modules():
#     # print(name, layer.__class__.__name__)


#     total_layers += 1

# print(f'Total number of layers: {total_layers}')

# conv_layers = 0
# for name, layer in model_0.named_modules():
#     if isinstance(layer, torch.nn.Sequential):
#         print(name, layer.__class__.__name__)
#         conv_layers += 1

# print(f'Total number of Sequential layers: {conv_layers}')

### 建立輸出為不同隱藏層的model

In [17]:
from helperFunction.helperFunctions import createDetailLayerVersions 
# detail version

list_of_models = createDetailLayerVersions(model_0)

# block level version

# list_of_models = []

# layer = 10
# list_of_models.append((model_0 , "layer:"+str(layer)))
# model = model_0

# while layer > 0:
#     model =  torch.nn.Sequential(*(list(model.children())[:-1])) 
#     layer -= 1
#     list_of_models.append((model , "layer:"+str(layer)))

# print(list_of_models)

# len(list(model.children()))

總層數為: 44層


In [18]:
# @ unit test : eval ability of model, in list of models
# summary(model_0, input_size=[1,3,224,224])
# summary(list_of_models[67][0] , input_size=[1,3,224,224])
# summary(list_of_models[66][0] , input_size=[1,3,224,224])
# summary(list_of_models[0][0] , input_size=[1,3,224,224])

In [19]:
# @ unit test : dataloaerToFeatureData()

# # model.forward( {data in datalaoder} ) -> (features , label)
# from helperFunction.helperFunctions import dataloaderToFeatureData

# model_0 = torch.nn.Sequential(*(list(model_0.children())[:-1])) 

# test_features , test_labels = dataloaderToFeatureData(model_0, test_dataloader,device)
# train_features , train_labels = dataloaderToFeatureData(model_0, train_dataloader, device)



In [20]:
# @ model test : 確認載入的model性能與原本相符

from helperFunction.TrainHelper import TrainingHelper

train_helper = TrainingHelper(model_0,
                              train_dataloader=train_dataloader,
                              test_dataloader=test_dataloader,
                              loss_fn=nn.CrossEntropyLoss(),
                              optimizer=torch.optim.SGD(model_0.parameters(), lr = 0.001),
                              device=device)

print("(loss, acc, f1 score):")
print(train_helper.test_step())


(loss, acc, f1 score):


c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(0.5040253672807936, 0.8299916457811195, 0.8196058895986511)


In [21]:
# @ unit test : model versions eval ability
summary(list_of_models[1][0], input_size=[1,3,224,224])
# print(list_of_models[1][0])
# print(list_of_models[0][0])
# type(list_of_models[0][0])
# type(list_of_models[1][0])

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 4096]                 --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

In [22]:
# @ testing : CNN model motification

# from other_program.custom_model.VGG16 import VGG16
# import inspect

# model_0dot0 = VGG16()

# list_of_dummy_model = createDetailLayerVersions(model_0dot0)

# summary(list_of_dummy_model[1][0], [1,3,224,224])

# for i in range(calDetailModelLayersNum(model_0dot0)):
#     try : 
#         summary(list_of_dummy_model[i][0], input_size=[1,3,224,224],depth=4)
#     except:
#         print("mat error : {} layer".format(i))

In [23]:
# print(list_of_models[67])

## 測試每層所需要的時間

In [24]:
# @  calModelEvalTime

# from helperFunction.helperFunctions import calModelEvalTime

# # eval_time_res = calModelEvalTime([(model_0, "layer0")], device)
# eval_time_res = calModelEvalTime(list_of_models, device)

# # 將字典轉換為DataFrame，但這次是轉置後的形式
# df_transposed = pd.DataFrame.from_dict(eval_time_res, orient='index').transpose()

# # 將轉置後的DataFrame存儲為CSV檔案
# csv_file_path_transposed = "eval_time_result_" + model_0_name + "_"+ device +  ".csv"
# df_transposed.to_csv(csv_file_path_transposed, index=False)

# asyncio.run(async_discord_bot_notifier("model eval time 計算完成!"))

## 測試每層所佔的容量

In [25]:
import torch

# 定义函数计算模型参数大小
def calculate_model_size(model):
    total_params = sum(p.numel() for p in model.parameters())
    bytes_per_param = next(model.parameters()).element_size()
    total_size_bytes = total_params * bytes_per_param
    total_size_megabytes = total_size_bytes / (1024 ** 2)  # 转换为兆字节
    return total_size_megabytes



# 计算并打印每个模型的内存占用
for idx, model in enumerate(list_of_models):
    try:
        model_size = calculate_model_size(model[0])
    except:
        break
    print(f"model name : {model[1]}")
    print(f"Model {idx+1} Size: {model_size:.2f} MB")



model name : layer:44
Model 1 Size: 512.27 MB
model name : layer:43
Model 2 Size: 512.16 MB
model name : layer:42
Model 3 Size: 512.16 MB
model name : layer:41
Model 4 Size: 512.16 MB
model name : layer:40
Model 5 Size: 448.15 MB
model name : layer:39
Model 6 Size: 448.15 MB
model name : layer:38
Model 7 Size: 448.15 MB
model name : layer:37
Model 8 Size: 56.13 MB
model name : layer:36
Model 9 Size: 56.13 MB
model name : layer:35
Model 10 Size: 56.13 MB
model name : layer:34
Model 11 Size: 56.13 MB
model name : layer:33
Model 12 Size: 56.13 MB
model name : layer:32
Model 13 Size: 56.13 MB
model name : layer:31
Model 14 Size: 56.13 MB
model name : layer:30
Model 15 Size: 47.13 MB
model name : layer:29
Model 16 Size: 47.13 MB
model name : layer:28
Model 17 Size: 38.13 MB
model name : layer:27
Model 18 Size: 38.13 MB
model name : layer:26
Model 19 Size: 29.13 MB
model name : layer:25
Model 20 Size: 29.13 MB
model name : layer:24
Model 21 Size: 29.13 MB
model name : layer:23
Model 22 Size:

In [26]:
# # @ test unit : check if muti modal function is ok

# test_features , test_labels = dataloaderToFeatureData(model_0, test_dataloader,device)


# test_muti_modal_vectors = []
# for idx , data in enumerate(test_features):
#     test_muti_modal_vectors.append(np.concatenate((test_features[idx] , test_img_feature_vector[idx])))
# test_features = np.array(test_muti_modal_vectors)

# nan_indices = np.where(np.isnan(test_features))
# print(f'Indices of NaN values: {list(zip(nan_indices[0], nan_indices[1]))}')
# print(test_features[1])
# print(test_features[0])

### 記錄下不同layer的輸出接到xgb的結果

In [27]:
# train_list, test_list = dataContainer.getFilenamesList()

# len(train_list), len(test_list)

In [28]:
import xgboost as xgb
print(xgb.config_context())

In [29]:
from helperFunction.helperFunctions import flattenTensor
# print(recordXGBoutput(model_0, train_dataloader, test_dataloader, "resnet", {}))

enable_FPGA_format_XGB = True
# enable_FPGA_format_XGB = False

XGB_res = {"model_name":[],
       "output_size":[],
       "num_parm":[],
       "acc":[],
       "f1":[],
       "iters":[],
       "CNN eval time":[],
       "XGB eval time":[],
       "CNN size":[],
       "XGB size":[],
       }

if enable_FPGA_format_XGB:
    XGB_res["f_acc"] = []
    XGB_res["f_f1"] = []
    XGB_res["f_iter"] = []
    XGB_res["f_XGB_size"] = []

layer_cnt = 0

# list_of_models = list_of_models[0:2]

for model, model_name in list_of_models:
    model = model.to(device)
    model_name = model_name.replace(":","")
    xgb_model_folder = model_folder_path + "/XGB/"
    xgb_output_model_name = data_select_signal + "_" + model_select_signal + "_"  +  model_name 
    xgb_model_path =  xgb_model_folder + xgb_output_model_name
    
    print(xgb_model_path)
    input_size = ""
    # 獲取model的output size
    with torch.no_grad():
        try:
            dummy_output = model(torch.rand([1, 3, 224, 224]).to(device))
        except RuntimeError:
            print("mat mismatch!!")
            continue
        except MemoryError:
            print("Not enough memory!! escape form loop")
            break
        except Exception as e:
            print("error catch :{}".format(e))
            break
        
        input_size = str(dummy_output.shape)[11:-1]

    print("input size is :{}".format(input_size))

    # training XGB !!!
    try:
        XGB_result = recordXGBoutput(model, train_dataloader, test_dataloader,enable_muti_module=enable_muti_modal_signal, model_output_path=xgb_model_path, FPGA_format=enable_FPGA_format_XGB)
    except MemoryError:
        print("Not enough memory!! escape form loop")
        break
    except RuntimeError:
        print("mat mismatch!!")
        continue
    # except:
    #     print("error catch ")
    #     break

    iter = XGB_result['iter']

    print("best acc is : {}".format(XGB_result['acc'][iter]))
    print("best f1 is : {}".format(XGB_result['f1'][iter]))
    print("XGB eval time : {}".format(XGB_result['XGB eval time']) )

    # print("model_name : {}".format(model_name) )
    # cal output size
    
    XGB_res["model_name"].append(model_name)
    XGB_res["output_size"].append(input_size)
    XGB_res["num_parm"].append(str(flattenTensor(dummy_output).shape)[11:-1])

    cur_acc = XGB_result['acc'][iter]
    XGB_res["acc"].append((-cur_acc) if cur_acc < 0 else cur_acc)
    cur_f1 = XGB_result['f1'][iter]
    XGB_res["f1"].append((-cur_f1) if cur_f1 < 0 else cur_f1)
    XGB_res["iters"].append(iter)

    XGB_res["CNN eval time"].append(XGB_result['CNN eval time'])
    XGB_res["XGB eval time"].append(XGB_result['XGB eval time'])
    XGB_res["CNN size"].append(XGB_result['CNN size'])
    XGB_res["XGB size"].append(XGB_result['XGB size'])

    if enable_FPGA_format_XGB:
        f_iter = XGB_result['f_iter']
        XGB_res["f_iter"].append(f_iter)
        XGB_res["f_acc"].append(XGB_result['f_acc'][f_iter])
        XGB_res["f_f1"].append(XGB_result['f_f1'][f_iter])
        XGB_res["f_XGB_size"].append(XGB_result['f_XGB_size'])




# 將字典轉換為DataFrame，但這次是轉置後的形式
df_transposed = pd.DataFrame.from_dict(XGB_res, orient='index').transpose()

# 將轉置後的DataFrame存儲為CSV檔案
csv_file_path_transposed = "XGB_model_results_" + model_0_name + ".csv"
df_transposed.to_csv(csv_file_path_transposed, index=False)

# 返回CSV檔案的儲存路徑
csv_file_path_transposed

asyncio.run(
    async_discord_bot_notifier("XGB訓練完成!")
)

../model/HAM10000/XGB/skin_vgg16_layer44
input size is :[1, 7]
cal CNN model output...
feature size is :2003
label size is :2003
feature size is :8012
label size is :8012
feature size is : (2003, 7)
[01:56:23] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\data\simple_dmatrix.cc:137: Generating new Ellpack page.
res's len : 2003
feature's len : 2003
跑測試資料的時間:8.406842225549732e-08
['f1', 'f4', 'f6', 'f0', 'f3', 'f5', 'f2']
[('f1', 789.0), ('f4', 740.0), ('f6', 723.0), ('f0', 693.0), ('f3', 672.0), ('f5', 623.0), ('f2', 566.0)]
[01:56:26] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\data\simple_dmatrix.cc:137: Generating new Ellpack page.
======eval finish!=========
best acc is : -0.817773
best f1 is : -0.811464
XGB eval time : 0.0001683890497777611
../model/HAM10000/XGB/skin_vgg16_layer43
input size is :[1, 4096]
cal CNN model outp

KeyboardInterrupt: 

### 紀錄其他ML模型的結果

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from helperFunction.helperFunctions import flattenTensor
# print(recordXGBoutput(model_0, train_dataloader, test_dataloader, "resnet", {}))

# clf setting 

# csv_model_name , clf = "Linear" , LogisticRegression()
# csv_model_name , clf = "SVC" , SVC(kernel='rbf',gamma='auto', probability=True)
csv_model_name , clf = "RandomForest" , RandomForestClassifier(n_estimators=100)
# csv_model_name , clf = "XGB105" , xgb.XGBClassifier()



linear_res = {"model_name":[],
       "output_size":[],
       "num_parm":[],
       "acc":[],
       "f1":[],
       "eval_time":[]}


layer_cnt = 0

for model, model_name in list_of_models:
    input_size = ""
    with torch.no_grad():
        try:
            dummy_output = model(torch.rand([1, 3, 244, 244]).to(device))
        except:
            print("ERROR!!!")
            break
        input_size = str(dummy_output.shape)[11:-1]

    print("input size is :{}".format(input_size))

    with torch.no_grad():
        # try:
        f1, acc = recordMLoutput(clf, model, train_dataloader, test_dataloader,enable_muti_module=enable_muti_modal_signal)
        # except:
        #     print("ERROR!!!")
        #     break


    # cal output size
    
    linear_res["model_name"].append(model_name)
    linear_res["output_size"].append(input_size)
    linear_res["num_parm"].append(str(flattenTensor(dummy_output).shape)[11:-1])
    
    linear_res["acc"].append(acc)
    linear_res["f1"].append(f1)

    layer_cnt += 1
    if layer_cnt >= 15:
        break

    print(model_name)



# 將字典轉換為DataFrame，但這次是轉置後的形式
df_transposed = pd.DataFrame.from_dict(linear_res, orient='index').transpose()

# 將轉置後的DataFrame存儲為CSV檔案
csv_file_path_transposed = csv_model_name + "_model_results_" +  model_select_signal + ".csv"
df_transposed.to_csv(csv_file_path_transposed, index=False)

# 返回CSV檔案的儲存路徑
csv_file_path_transposed

asyncio.run(async_discord_bot_notifier("ML model訓練完成!"))

input size is :[1, 7]
cal CNN model output...


KeyboardInterrupt: 